In [1]:
import pyNeuroChem as pyc
import hdnntools as hdt
import numpy as np
import time

from ase_interface import ANIENS
from ase_interface import ensemblemolecule

from ase.atoms import Atoms
from ase.optimize import BFGS, LBFGS

import pyaniasetools as pya

import batch_optimizers as opt

van Der Waals correction will be unavailable. Please install ased3


In [2]:
mol = hdt.readxyz2('/nh/nest/u/jsmith/Research/confs_test/acetaminophen.xyz')
N = 250

wkdir = '/nh/nest/u/jsmith/Research/confs_test/arc9/'
cns = wkdir + 'rHCNO-4.6R_16-3.1A_a4-8.params'
sae = wkdir + 'sae_linfit.dat'
nnf = wkdir + 'train'
Nn = 2

In [3]:
Xl = np.stack([mol[0][0] for i in range(N)])
S = list(mol[1])
Xl.shape

(250, 20, 3)

In [4]:
X = Xl+np.random.uniform(-0.15,0.15,size=Xl.shape)
X = np.asarray(X, dtype=np.float32)
hdt.writexyzfile('/nh/nest/u/jsmith/Research/confs_test/verify1.xyz',X,S)

In [5]:
# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 0)

In [7]:
start_time = time.time()
for i,x in enumerate(X):
    mol = Atoms(S,x)
    mol.set_calculator(ANIENS(aens))
    
    dyn = LBFGS(mol, logfile='log.out')
    dyn.run(fmax=0.00018)
    print('Molecule',i,'complete')
print('[ANI Total time:', time.time() - start_time, 'seconds]')

Molecule 0 complete
Molecule 1 complete
Molecule 2 complete
Molecule 3 complete
Molecule 4 complete
Molecule 5 complete
Molecule 6 complete
Molecule 7 complete
Molecule 8 complete
Molecule 9 complete
Molecule 10 complete
Molecule 11 complete
Molecule 12 complete
Molecule 13 complete
Molecule 14 complete
Molecule 15 complete
Molecule 16 complete
Molecule 17 complete
Molecule 18 complete
Molecule 19 complete
Molecule 20 complete
Molecule 21 complete
Molecule 22 complete
Molecule 23 complete
Molecule 24 complete
Molecule 25 complete
Molecule 26 complete
Molecule 27 complete
Molecule 28 complete
Molecule 29 complete
Molecule 30 complete
Molecule 31 complete
Molecule 32 complete
Molecule 33 complete
Molecule 34 complete
Molecule 35 complete
Molecule 36 complete
Molecule 37 complete
Molecule 38 complete
Molecule 39 complete
Molecule 40 complete
Molecule 41 complete
Molecule 42 complete
Molecule 43 complete
Molecule 44 complete
Molecule 45 complete
Molecule 46 complete
Molecule 47 complete
Mo

In [8]:
molopt = opt.moleculeOptimizer(cns, sae, nnf, Nn, 0)

In [9]:
start_time = time.time()
X_it = molopt.optimizeGradientDescent(X,S,alpha=0.0004,convergence=1.0, printer=False)
X_it = molopt.optimizeConjugateGradient(X_it,S,alpha=0.0001,convergence=0.005, maxsteps=5000, printstep=100, printer=False)
print('[Complete Optimization Time:',time.time()-start_time,'s]')
hdt.writexyzfile('/nh/nest/u/jsmith/Research/confs_test/verify_mol1.xyz',np.array(X_it),S)

--Optimizing conformation: 0 --
Complete
  - 146 -323383.371 16.9256 0.9962 0.2515
--Optimizing conformation: 1 --
Complete
  - 194 -323383.724 13.1054 0.9934 0.2646
--Optimizing conformation: 2 --
Complete
  - 142 -323383.316 19.4394 0.9951 0.2366
--Optimizing conformation: 3 --
Complete
  - 175 -323383.618 14.2665 0.9978 0.2523
--Optimizing conformation: 4 --
Complete
  - 150 -323383.619 14.0194 0.9985 0.2694
--Optimizing conformation: 5 --
Complete
  - 104 -323383.610 17.4148 0.9989 0.2474
--Optimizing conformation: 6 --
Complete
  - 142 -323383.683 12.4151 0.9930 0.2468
--Optimizing conformation: 7 --
Complete
  - 153 -323383.549 13.4662 0.9954 0.2538
--Optimizing conformation: 8 --
Complete
  - 144 -323383.702 13.0415 0.9955 0.2544
--Optimizing conformation: 9 --
Complete
  - 138 -323383.629 16.4812 0.9910 0.2565
--Optimizing conformation: 10 --
Complete
  - 157 -323383.814 12.8764 0.9924 0.2598
--Optimizing conformation: 11 --
Complete
  - 148 -323383.302 19.8571 0.9985 0.2414
--

  - 145 -323383.452 14.2381 0.9937 0.2414
--Optimizing conformation: 102 --
Complete
  - 101 -323383.159 21.7535 0.9947 0.2336
--Optimizing conformation: 103 --
Complete
  - 148 -323383.563 13.7360 0.9969 0.2501
--Optimizing conformation: 104 --
Complete
  - 164 -323383.332 16.5747 0.9985 0.2409
--Optimizing conformation: 105 --
Complete
  - 129 -323383.353 18.4517 0.9950 0.2409
--Optimizing conformation: 106 --
Complete
  - 140 -323383.408 14.9048 0.9961 0.2369
--Optimizing conformation: 107 --
Complete
  - 127 -323383.318 19.7471 0.9945 0.2449
--Optimizing conformation: 108 --
Complete
  - 180 -323383.587 13.5779 0.9935 0.2520
--Optimizing conformation: 109 --
Complete
  - 146 -323383.537 15.0935 0.9965 0.2449
--Optimizing conformation: 110 --
Complete
  - 161 -323383.510 13.8780 0.9944 0.2442
--Optimizing conformation: 111 --
Complete
  - 150 -323383.453 14.3537 0.9961 0.2425
--Optimizing conformation: 112 --
Complete
  - 118 -323383.541 16.0632 0.9993 0.2539
--Optimizing conformati

--Optimizing conformation: 201 --
Complete
  - 157 -323383.747 13.6751 0.9968 0.2610
--Optimizing conformation: 202 --
Complete
  - 142 -323383.556 15.3246 0.9976 0.2527
--Optimizing conformation: 203 --
Complete
  - 173 -323383.504 13.7948 0.9973 0.2455
--Optimizing conformation: 204 --
Complete
  - 143 -323383.594 11.7802 0.9930 0.2526
--Optimizing conformation: 205 --
Complete
  - 138 -323383.605 14.6877 0.9933 0.2539
--Optimizing conformation: 206 --
Complete
  - 74 -323383.527 17.7832 0.9957 0.2470
--Optimizing conformation: 207 --
Complete
  - 156 -323383.642 15.6765 0.9940 0.2528
--Optimizing conformation: 208 --
Complete
  - 186 -323383.551 13.6193 0.9962 0.2619
--Optimizing conformation: 209 --
Complete
  - 165 -323383.291 16.9198 0.9968 0.2430
--Optimizing conformation: 210 --
Complete
  - 130 -323383.201 19.3737 0.9949 0.2395
--Optimizing conformation: 211 --
Complete
  - 141 -323383.766 13.8139 0.9960 0.2581
--Optimizing conformation: 212 --
Complete
  - 139 -323383.813 10.

Complete
  - 807 -323384.604 0.0755 0.0049 0.2979
--Optimizing conformation: 51 --
Complete
  - 759 -323384.602 0.1096 0.0050 0.2981
--Optimizing conformation: 52 --
Complete
  - 786 -323384.603 0.0734 0.0049 0.2980
--Optimizing conformation: 53 --
Complete
  - 762 -323384.599 0.1126 0.0049 0.2983
--Optimizing conformation: 54 --
Complete
  - 807 -323384.602 0.0715 0.0049 0.2982
--Optimizing conformation: 55 --
Complete
  - 828 -323384.607 0.0928 0.0049 0.2975
--Optimizing conformation: 56 --
Complete
  - 546 -323384.599 0.0986 0.0049 0.2987
--Optimizing conformation: 57 --
Complete
  - 774 -323384.603 0.1313 0.0048 0.2980
--Optimizing conformation: 58 --
Complete
  - 831 -323384.603 0.0806 0.0050 0.2982
--Optimizing conformation: 59 --
Complete
  - 792 -323384.603 0.0793 0.0049 0.2982
--Optimizing conformation: 60 --
Complete
  - 760 -323384.602 0.1346 0.0049 0.2979
--Optimizing conformation: 61 --
Complete
  - 755 -323384.604 0.0626 0.0048 0.2981
--Optimizing conformation: 62 --
Comp

Complete
  - 779 -323384.600 0.0895 0.0050 0.2983
--Optimizing conformation: 150 --
Complete
  - 853 -323384.600 0.0718 0.0050 0.2984
--Optimizing conformation: 151 --
Complete
  - 744 -323384.600 0.0901 0.0049 0.2981
--Optimizing conformation: 152 --
Complete
  - 779 -323384.602 0.0719 0.0050 0.2981
--Optimizing conformation: 153 --
Complete
  - 788 -323384.601 0.0741 0.0050 0.2983
--Optimizing conformation: 154 --
Complete
  - 777 -323384.601 0.0982 0.0048 0.2983
--Optimizing conformation: 155 --
Complete
  - 744 -323384.603 0.0652 0.0050 0.2981
--Optimizing conformation: 156 --
Complete
  - 770 -323384.601 0.1085 0.0049 0.2982
--Optimizing conformation: 157 --
Complete
  - 763 -323384.601 0.0934 0.0049 0.2982
--Optimizing conformation: 158 --
Complete
  - 788 -323384.601 0.0933 0.0050 0.2983
--Optimizing conformation: 159 --
Complete
  - 796 -323384.604 0.0875 0.0049 0.2979
--Optimizing conformation: 160 --
Complete
  - 771 -323384.603 0.0769 0.0049 0.2981
--Optimizing conformation:

Complete
  - 795 -323384.603 0.0913 0.0049 0.2980
--Optimizing conformation: 248 --
Complete
  - 766 -323384.602 0.1161 0.0050 0.2981
--Optimizing conformation: 249 --
Complete
  - 776 -323384.603 0.0837 0.0048 0.2979
[Complete Optimization Time: 420.99644231796265 s]


In [10]:
cnfopt = opt.conformerOptimizer(cns, sae, nnf, Nn, 0)

In [11]:
start_time = time.time()
X_it = cnfopt.optimizeGradientDescent(X,S,alpha=0.0004,convergence=1.0)
X_it = cnfopt.optimizeConjugateGradient(X_it,S,alpha=0.0001,convergence=0.005, maxsteps=1000, printstep=100)
print('[Complete Optimization Time:',time.time()-start_time,'s]')
hdt.writexyzfile('/nh/nest/u/jsmith/Research/confs_test/verify_mol2.xyz',np.array(X_it),S)

--Optimizing all conformations--
  - 0000 ) 0 of 250 4897.8164 335.6249 11.0047
  - 0050 ) 0 of 250 47.7586 3.1376 0.1000
  - 0100 ) 11 of 250 22.8277 1.5286 0.0512
  - 0150 ) 156 of 250 16.7740 1.0643 0.0334
  - 0200 ) 248 of 250 16.0868 0.9961 0.0046
Complete
  - 0207 ) 250 of 250 16.0846 0.9959 0.0000
--Optimizing all conformations--
  - 0000 ) 0 of 250 15.9667 0.9875 0.0354
  - 0100 ) 0 of 250 5.1412 0.3476 0.0140
  - 0200 ) 0 of 250 2.2668 0.1494 0.0038
  - 0300 ) 0 of 250 0.8946 0.0577 0.0025
  - 0400 ) 0 of 250 0.3395 0.0257 0.0018
  - 0500 ) 0 of 250 0.3257 0.0201 0.0011
  - 0600 ) 1 of 250 0.2793 0.0172 0.0005
  - 0700 ) 1 of 250 0.1526 0.0100 0.0004
  - 0800 ) 200 of 250 0.0954 0.0051 0.0005
Complete
  - 0859 ) 250 of 250 0.0924 0.0049 0.0000
[Complete Optimization Time: 12.15681791305542 s]
